In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [ ]:
def preprocess_sensor_data(data_path):
    """
    Preprocess accelerometer data for gait analysis.
    Args:
        data_path (str): Path to the dataset (CSV file).
    Returns:
        X_train, X_test, y_train, y_test: Processed training and test sets.
    """
    # Load the dataset
    data = pd.read_csv(data_path)
    
    # Drop missing values
    data.dropna(inplace=True)
    
    # Normalize accelerometer readings (x, y, z)
    scaler = StandardScaler()
    data[['x', 'y', 'z']] = scaler.fit_transform(data[['x', 'y', 'z']])
    
    # Create feature windows (e.g., 3 seconds with 50 Hz sampling rate -> 150 samples/window)
    window_size = 150
    features, labels = [], []
    for i in range(0, len(data) - window_size, window_size):
        window = data.iloc[i:i + window_size]
        features.append(window[['x', 'y', 'z']].values.flatten())
        labels.append(window['activity_label'].mode()[0])  # Majority vote for label
    
    # Convert to NumPy arrays
    X = np.array(features)
    y = np.array(labels)
    
    # Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

# Example usage
X_train, X_test, y_train, y_test = preprocess_sensor_data("gait_data.csv")


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

def train_lstm_model(X_train, y_train, X_test, y_test):
    """
    Train an LSTM model for gait analysis.
    """
    # Reshape input to 3D for LSTM (samples, timesteps, features)
    timesteps = 150  # Window size
    features = 3  # x, y, z axes
    X_train = X_train.reshape(-1, timesteps, features)
    X_test = X_test.reshape(-1, timesteps, features)

    # Encode labels (if categorical)
    y_train = tf.keras.utils.to_categorical(y_train)
    y_test = tf.keras.utils.to_categorical(y_test)
    
    # Define LSTM model
    model = Sequential([
        LSTM(64, input_shape=(timesteps, features), return_sequences=True),
        Dropout(0.3),
        LSTM(64),
        Dropout(0.3),
        Dense(y_train.shape[1], activation='softmax')  # Number of classes
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)
    
    return model

# Train the model
model = train_lstm_model(X_train, y_train, X_test, y_test)


In [ ]:
from flask import Flask, request, jsonify
import numpy as np

app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    """
    Predict if the incoming gait data matches an employee.
    """
    data = request.get_json()
    sensor_data = np.array(data['sensor_data']).reshape(1, 150, 3)  # Reshape for LSTM
    prediction = model.predict(sensor_data)
    employee_id = np.argmax(prediction)
    return jsonify({"employee_id": int(employee_id)})

if __name__ == '__main__':
    app.run(debug=True)
